In [2]:
!pip install ipywidgets matplotlib --quiet

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider, Button
from IPython.display import display, clear_output

# --- Αρχικοποίηση παραμέτρων ---
M = 50  # Διαστάσεις πλέγματος (MxM)

# Δυνατότητες του εδάφους (χωρητικότητα)
CN, CP, CK = 1.0, 1.0, 1.0  # max concentration

# Ρυθμοί διάχυσης
DN, DP, DK = 0.1, 0.05, 0.07

# Θερμοκρασία και υγρασία σταθερές
Ts = 25  # Celsius
Hs = 0.6  # 0 to 1

# --- Δημιουργία πλέγματος ---
def initialize_grid(granules=50):
    grid = np.zeros((M, M, 5))  # [cn, cp, ck, hs, ts]
    grid[:, :, 3] = Hs
    grid[:, :, 4] = Ts
    
    for _ in range(granules):
        x, y = np.random.randint(0, M), np.random.randint(0, M)
        grid[x, y, 0] += 0.5  # N
        grid[x, y, 1] += 0.5  # P
        grid[x, y, 2] += 0.5  # K
    return grid

# --- Διάχυση με χρήση μέσης τιμής γειτονικών κυψελίδων ---
def diffuse(grid, steps=10):
    for _ in range(steps):
        new_grid = grid.copy()
        for i in range(1, M-1):
            for j in range(1, M-1):
                for k, D in enumerate([DN, DP, DK]):
                    neighborhood = grid[i-1:i+2, j-1:j+2, k]
                    diffusion = D * (np.mean(neighborhood) - grid[i, j, k])
                    new_grid[i, j, k] += diffusion
                    new_grid[i, j, k] = min(new_grid[i, j, k], [CN, CP, CK][k])
        grid = new_grid
    return grid

# --- Οπτικοποίηση διάχυσης ---
def plot_diffusion(grid):
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    labels = ['N (Azote)', 'P (Phosphorus)', 'K (Potassium)']
    for i in range(3):
        axs[i].imshow(grid[:, :, i], cmap='viridis')
        axs[i].set_title(labels[i])
        axs[i].axis('off')
    plt.show()

# --- GUI με sliders ---
def run_simulation(granules, steps, Dn, Dp, Dk):
    global DN, DP, DK
    DN, DP, DK = Dn, Dp, Dk
    grid = initialize_grid(granules)
    final = diffuse(grid, steps)
    plot_diffusion(final)

# GUI controls
interact(
    run_simulation,
    granules=IntSlider(min=10, max=200, step=10, value=50, description='Κόκκοι'),
    steps=IntSlider(min=1, max=50, step=1, value=10, description='Βήματα'),
    Dn=FloatSlider(min=0.01, max=0.3, step=0.01, value=0.1, description='Διάχυση N'),
    Dp=FloatSlider(min=0.01, max=0.3, step=0.01, value=0.05, description='Διάχυση P'),
    Dk=FloatSlider(min=0.01, max=0.3, step=0.01, value=0.07, description='Διάχυση K')
);


interactive(children=(IntSlider(value=50, description='Κόκκοι', max=200, min=10, step=10), IntSlider(value=10,…

In [4]:
!pip install ipywidgets matplotlib --quiet

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

# --- Παράμετροι μοντέλου ---
M = 50  # Μέγεθος πλέγματος
CN, CP, CK = 1.0, 1.0, 1.0  # Χωρητικότητα εδάφους
DN, DP, DK = 0.1, 0.05, 0.07  # Συντελεστές διάχυσης
Hs, Ts = 0.6, 25  # Σταθερή υγρασία & θερμοκρασία

# --- Αρχικοποίηση πλέγματος ---
def initialize_grid(granules=50):
    grid = np.zeros((M, M, 5))  # [cn, cp, ck, hs, ts]
    grid[:, :, 3] = Hs
    grid[:, :, 4] = Ts

    for _ in range(granules):
        x, y = np.random.randint(0, M), np.random.randint(0, M)
        grid[x, y, 0] += 0.5
        grid[x, y, 1] += 0.5
        grid[x, y, 2] += 0.5
    return grid

# --- Διάχυση για ένα βήμα ---
def diffuse_step(grid):
    new_grid = grid.copy()
    for i in range(1, M - 1):
        for j in range(1, M - 1):
            for k, D in enumerate([DN, DP, DK]):
                neighborhood = grid[i-1:i+2, j-1:j+2, k]
                diffusion = D * (np.mean(neighborhood) - grid[i, j, k])
                new_grid[i, j, k] += diffusion
                new_grid[i, j, k] = min(new_grid[i, j, k], [CN, CP, CK][k])
    return new_grid

# --- Δημιουργία animation ---
def create_animation(granules=80, steps=30):
    grid = initialize_grid(granules)
    frames = []

    for _ in range(steps):
        grid = diffuse_step(grid)
        frame = grid[:, :, :3].copy()  # μόνο NPK
        frames.append(frame)

    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    ims = []

    for i in range(len(frames)):
        ims_frame = []
        for j in range(3):
            im = axes[j].imshow(frames[i][:, :, j], cmap='viridis', animated=True)
            axes[j].set_title(['Azote (N)', 'Phosphorus (P)', 'Potassium (K)'][j])
            axes[j].axis('off')
            ims_frame.append(im)
        ims.append(ims_frame)

    ani = animation.ArtistAnimation(fig, ims, interval=300, blit=True, repeat=False)
    plt.close()
    return HTML(ani.to_jshtml())

# --- Εκτέλεση animation ---
create_animation(granules=100, steps=40)


In [5]:
!pip install ipywidgets matplotlib --quiet

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML, display
import ipywidgets as widgets

# --- Παράμετροι μοντέλου ---
M = 50
CN, CP, CK = 1.0, 1.0, 1.0

# Βασικοί ρυθμοί διάχυσης
base_DN, base_DP, base_DK = 0.1, 0.05, 0.07

# Τύποι εδάφους και η υγρασία τους
soil_types = {
    'Ξηρό': 0.2,
    'Μέτριο': 0.5,
    'Υγρό': 0.8
}

def initialize_grid(granules=50, hs=0.5):
    grid = np.zeros((M, M, 5))  # [cn, cp, ck, hs, ts]
    grid[:, :, 3] = hs
    grid[:, :, 4] = 25  # Σταθερή θερμοκρασία

    for _ in range(granules):
        x, y = np.random.randint(0, M), np.random.randint(0, M)
        grid[x, y, 0] += 0.5
        grid[x, y, 1] += 0.5
        grid[x, y, 2] += 0.5
    return grid

def diffuse_step(grid, DN, DP, DK):
    new_grid = grid.copy()
    for i in range(1, M - 1):
        for j in range(1, M - 1):
            local_hs = grid[i, j, 3]
            damp = 0.5 + local_hs  # Υγρασία επηρεάζει τη διάχυση
            for k, base_D in enumerate([DN, DP, DK]):
                D = base_D * damp
                neighborhood = grid[i-1:i+2, j-1:j+2, k]
                diffusion = D * (np.mean(neighborhood) - grid[i, j, k])
                new_grid[i, j, k] += diffusion
                new_grid[i, j, k] = min(new_grid[i, j, k], [CN, CP, CK][k])
    return new_grid

def create_animation(granules, steps, DN, DP, DK, soil_type):
    hs = soil_types[soil_type]
    grid = initialize_grid(granules, hs)
    frames = []

    for _ in range(steps):
        grid = diffuse_step(grid, DN, DP, DK)
        frames.append(grid[:, :, :3].copy())

    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    ims = []

    for i in range(len(frames)):
        ims_frame = []
        for j in range(3):
            im = axes[j].imshow(frames[i][:, :, j], cmap='viridis', animated=True, vmin=0, vmax=1)
            axes[j].set_title(['Azote (N)', 'Phosphorus (P)', 'Potassium (K)'][j])
            axes[j].axis('off')
            ims_frame.append(im)
        ims.append(ims_frame)

    ani = animation.ArtistAnimation(fig, ims, interval=300, blit=True, repeat=False)
    plt.close()
    return HTML(ani.to_jshtml())

# --- GUI Widgets ---
granules_slider = widgets.IntSlider(min=10, max=200, step=10, value=50, description='Κόκκοι')
steps_slider = widgets.IntSlider(min=5, max=60, step=5, value=30, description='Βήματα')
DN_slider = widgets.FloatSlider(min=0.01, max=0.3, step=0.01, value=0.1, description='Διάχυση N')
DP_slider = widgets.FloatSlider(min=0.01, max=0.3, step=0.01, value=0.05, description='Διάχυση P')
DK_slider = widgets.FloatSlider(min=0.01, max=0.3, step=0.01, value=0.07, description='Διάχυση K')
soil_dropdown = widgets.Dropdown(options=soil_types.keys(), value='Μέτριο', description='Έδαφος')

run_button = widgets.Button(description='Εκκίνηση Προσομοίωσης', button_style='success')

output = widgets.Output()

def on_button_clicked(b):
    with output:
        output.clear_output()
        display(create_animation(
            granules_slider.value,
            steps_slider.value,
            DN_slider.value,
            DP_slider.value,
            DK_slider.value,
            soil_dropdown.value
        ))

run_button.on_click(on_button_clicked)

# --- Εμφάνιση GUI ---
display(widgets.VBox([
    granules_slider, steps_slider,
    DN_slider, DP_slider, DK_slider,
    soil_dropdown,
    run_button, output
]))
